<a href="https://colab.research.google.com/github/RaghulKumar004/RaghulSwags/blob/main/loanprediction(intermediate).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

data_path = '/content/loan_prediction.csv'
data = pd.read_csv(data_path)

data.drop(columns=['Loan_ID'], inplace=True, errors='ignore')

def preprocess_data(df):
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())

    categorical_cols = df.select_dtypes(include=[object]).columns
    df[categorical_cols] = df[categorical_cols].apply(lambda x: x.fillna(x.mode()[0]))

    label_encoders = {}
    for col in categorical_cols:
        label_encoders[col] = LabelEncoder()
        df[col] = label_encoders[col].fit_transform(df[col])

    return df, label_encoders

data, label_encoders = preprocess_data(data)
X = data.drop('Loan_Status', axis=1)
y = data['Loan_Status']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = LogisticRegression()
model.fit(X_train, y_train)

def predict_loan_eligibility():
    applicant_data = {
        'Gender': input("Gender (Male/Female): "),
        'Married': input("Married (Yes/No): "),
        'Dependents': input("Dependents (0/1/2/3+): "),
        'Education': input("Education (Graduate/Not Graduate): "),
        'Self_Employed': input("Self Employed (Yes/No): "),
        'ApplicantIncome': float(input("Applicant Income: ")),
        'CoapplicantIncome': float(input("Coapplicant Income: ")),
        'LoanAmount': float(input("Loan Amount: ")),
        'Loan_Amount_Term': float(input("Loan Amount Term: ")),
        'Credit_History': float(input("Credit History (0 or 1): ")),
        'Property_Area': input("Property Area (Urban/Semiurban/Rural): ")
    }

    applicant_df = pd.DataFrame([applicant_data])

    missing_cols = set(X.columns) - set(applicant_df.columns)
    for col in missing_cols:
        applicant_df[col] = 0

    for col in label_encoders.keys():
        if col in applicant_df.columns:
            if applicant_df[col].iloc[0] in label_encoders[col].classes_:
                applicant_df[col] = label_encoders[col].transform(applicant_df[col])
            else:
                print(f"Warning: {applicant_df[col].iloc[0]} not seen before in column {col}.")
                applicant_df[col] = -1

    applicant_df = applicant_df[X.columns]
    applicant_df = scaler.transform(applicant_df)
    prediction = model.predict(applicant_df)

    return 'Approved' if prediction[0] == 1 else 'Rejected'

result = predict_loan_eligibility()
print(f'Loan Eligibility: {result}')


Gender (Male/Female): Male
Married (Yes/No): Yes
Dependents (0/1/2/3+): 2
Education (Graduate/Not Graduate): Graduate
Self Employed (Yes/No): Yes
Applicant Income: 10000
Coapplicant Income: 20000
Loan Amount: 100000
Loan Amount Term: 2000
Credit History (0 or 1): 0
Property Area (Urban/Semiurban/Rural): Urban
Loan Eligibility: Rejected
